In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'São Paulo - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [8]:
data = pd.read_csv('2003_01_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland,São Paulo - Consumo de Cimento (t)
0,2003,5232.682934,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,0.812723,1.034903e+09,4.730940e+07,23.895559,9.819129e+08,8.310335,3.922845,9528.051000
1,2004,5521.391778,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,0.814104,1.049168e+09,4.755584e+07,23.966513,9.861577e+08,8.236641,4.120177,9148.318000
2,2005,6102.302057,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,0.815359,1.062582e+09,4.779561e+07,24.046393,9.901728e+08,8.212526,4.124581,9724.722000
3,2006,6788.458005,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,0.816488,1.074248e+09,4.799886e+07,24.099559,9.927879e+08,8.126466,3.981310,10863.790000
4,2007,7337.194102,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,0.817495,1.084443e+09,4.833495e+07,24.140672,9.945643e+08,8.101259,3.847089,11775.453000
5,2008,7770.929330,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,0.818381,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,8.071263,3.773615,13476.268000
6,2009,8316.504885,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,0.819150,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8.100327,3.771810,13304.301000
7,2010,8835.509117,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,0.819803,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8.064867,3.913291,14534.980000
8,2011,9192.769768,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,0.820344,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,8.053175,4.060906,14871.943000
9,2012,9435.198835,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,0.820550,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,10.058484,4.219677,15159.683000


In [9]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
0,5232.682934,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,0.812723,1.034903e+09,4.730940e+07,23.895559,9.819129e+08,8.310335,3.922845
1,5521.391778,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,0.814104,1.049168e+09,4.755584e+07,23.966513,9.861577e+08,8.236641,4.120177
2,6102.302057,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,0.815359,1.062582e+09,4.779561e+07,24.046393,9.901728e+08,8.212526,4.124581
3,6788.458005,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,0.816488,1.074248e+09,4.799886e+07,24.099559,9.927879e+08,8.126466,3.981310
4,7337.194102,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,0.817495,1.084443e+09,4.833495e+07,24.140672,9.945643e+08,8.101259,3.847089
5,7770.929330,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,0.818381,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,8.071263,3.773615
6,8316.504885,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,0.819150,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8.100327,3.771810
7,8835.509117,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,0.819803,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8.064867,3.913291
8,9192.769768,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,0.820344,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,8.053175,4.060906
9,9435.198835,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,0.820550,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,10.058484,4.219677


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      9148.318000
1      9724.722000
2     10863.790000
3     11775.453000
4     13476.268000
5     13304.301000
6     14534.980000
7     14871.943000
8     15159.683000
9     15089.297000
10    13714.511286
11    11767.653429
12     9811.084286
13     9918.501000
14    10001.077000
15    10458.276000
16    11181.809000
17    12113.838000
18    12383.649000
19             NaN
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [11]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
0,-1.373010,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-2.115587,-1.653768,-0.358309,-1.337901,-2.477902,-0.810130,-0.887599
1,-1.170521,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.679161,-1.427561,-0.071270,-0.606200,-1.773047,-0.856132,-0.454655
2,-0.763092,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.282958,-1.214848,0.208010,0.217547,-1.106338,-0.871185,-0.444993
3,-0.281848,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.926225,-1.029862,0.444753,0.765810,-0.672087,-0.924906,-0.759327
4,0.103015,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.608210,-0.868197,0.836216,1.189782,-0.377108,-0.940640,-1.053806
5,0.407220,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.328158,-0.742092,1.064358,1.221901,-0.357446,-0.959365,-1.215007
6,0.789867,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.085317,-0.642613,1.300921,1.198844,-0.352980,-0.941222,-1.218968
7,1.153877,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.121064,-0.240784,1.432729,1.250915,0.280020,-0.963357,-0.908559
8,1.404446,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.291741,0.118022,1.082972,1.123993,0.651955,-0.970656,-0.584694
9,1.574477,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.356913,0.436508,0.725426,0.663733,0.888800,0.281103,-0.236354


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      9148.318000
1      9724.722000
2     10863.790000
3     11775.453000
4     13476.268000
5     13304.301000
6     14534.980000
7     14871.943000
8     15159.683000
9     15089.297000
10    13714.511286
11    11767.653429
12     9811.084286
13     9918.501000
14    10001.077000
15    10458.276000
16    11181.809000
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
17,-0.733293,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.064924,0.776026,-1.154968,-1.420169,0.021261,0.852494,2.586291


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    12113.838
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [15]:
train_input.iloc[-(len(train_input)//5):]

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
14,-1.049614,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.243424,1.189596,-1.504471,-1.181425,0.679759,1.172636,1.148630
15,-1.149965,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.023912,1.150795,-1.545321,-1.270370,0.550558,1.068127,1.412478
16,-0.995655,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,0.612137,1.039649,-1.465420,-1.334622,0.398702,0.992960,1.960645


In [16]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3255904894, 2728350327, 2453252687, 932487807, 1791310210, 988770512, 4204676213, 908615603, 3063914784, 3405536698, 3422720036, 1502925150, 3163806219, 1691404978, 519104088, 2185023923, 2139658711, 685769025, 340709390, 2399149786, 3993056742, 4223496056, 2935464336, 1283994143, 3754167286, 3959993199, 3625577693, 3564502937, 291310512, 3979459115, 1799248601, 109971379, 4128821118, 1265874657, 1019978933, 1370614552, 3014225030, 2662770737, 3883138029, 2031626188, 2741724557, 217011556, 4079897798, 3718141191, 2682728148, 1694012970, 4230832954, 3651749011, 2479972954, 524575767, 3788781758, 1309525098, 3626768885, 1720078132, 1547342868, 640392479, 223766338, 3384249503, 1228360687, 2512456243, 3804606211, 3147734926, 1795130778, 4262285292, 2944413193, 3247703001, 10497663, 1882508159, 849678210, 443432499, 620992560, 4280770695, 1898970041, 185855079, 4129737757, 3169698313, 1543529007, 1585901861, 3425178565, 1654905673, 4053038696, 2461083980, 2139856908, 708732543, 3508645057

val_loss: 21039.486328125


Step: 90 ___________________________________________
val_loss: 278913.71875


Step: 91 ___________________________________________
val_loss: 167686.453125


Step: 92 ___________________________________________
val_loss: 249847.25


Step: 93 ___________________________________________
val_loss: 232467.328125


Step: 94 ___________________________________________
val_loss: 28433.0390625


Step: 95 ___________________________________________
val_loss: 236300.25


Step: 96 ___________________________________________
val_loss: 6915.21826171875


Step: 97 ___________________________________________
val_loss: 145422.703125


Step: 98 ___________________________________________
val_loss: 44679.94140625


Step: 99 ___________________________________________
val_loss: 281096.84375


final_seed: 3255904894


In [21]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 447ms/step - loss: 149362208.0000 - val_loss: 110170336.0000
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 146402336.0000 - val_loss: 93276032.0000
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 147757184.0000 - val_loss: 109258792.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 113255288.0000 - val_loss: 217875504.0000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 122952312.0000 - val_loss: 23187678.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 116005840.0000 - val_loss: 163058544.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 112814376.0000 - val_loss: 35946720.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 99885400.0000 - val_loss: 15363536.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 11

1/1 [==============================] - 0s 14ms/step - loss: 38260896.0000 - val_loss: 40739268.0000
Epoch 73/10000
1/1 [==============================] - 0s 14ms/step - loss: 40271892.0000 - val_loss: 40873092.0000
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 34780260.0000 - val_loss: 34916688.0000
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 37321560.0000 - val_loss: 43828736.0000
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 36811244.0000 - val_loss: 32967814.0000
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 37157536.0000 - val_loss: 39081452.0000
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 29186916.0000 - val_loss: 35567036.0000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 28796496.0000 - val_loss: 31309046.0000
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 26794044.0000 - val_

1/1 [==============================] - 0s 14ms/step - loss: 10190872.0000 - val_loss: 11459524.0000
Epoch 144/10000
1/1 [==============================] - 0s 13ms/step - loss: 11213106.0000 - val_loss: 12752077.0000
Epoch 145/10000
1/1 [==============================] - 0s 13ms/step - loss: 9543136.0000 - val_loss: 12278647.0000
Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 9610609.0000 - val_loss: 20494118.0000
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 13220072.0000 - val_loss: 13106864.0000
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 12502252.0000 - val_loss: 13661695.0000
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 9487015.0000 - val_loss: 10127408.0000
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 9155419.0000 - val_loss: 10632211.0000
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 9270095.0000 - v

1/1 [==============================] - 0s 13ms/step - loss: 4815757.5000 - val_loss: 4681669.5000
Epoch 215/10000
1/1 [==============================] - 0s 14ms/step - loss: 3258041.7500 - val_loss: 4006136.0000
Epoch 216/10000
1/1 [==============================] - 0s 14ms/step - loss: 3176841.0000 - val_loss: 10901921.0000
Epoch 217/10000
1/1 [==============================] - 0s 13ms/step - loss: 6882980.0000 - val_loss: 3612606.0000
Epoch 218/10000
1/1 [==============================] - 0s 14ms/step - loss: 4357415.0000 - val_loss: 4990749.0000
Epoch 219/10000
1/1 [==============================] - 0s 13ms/step - loss: 10515013.0000 - val_loss: 2986426.0000
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 4664480.0000 - val_loss: 5595498.5000
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 3775556.5000 - val_loss: 2062927.5000
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 4824792.0000 - val_loss: 5

1/1 [==============================] - 0s 14ms/step - loss: 2116835.2500 - val_loss: 2014666.8750
Epoch 287/10000
1/1 [==============================] - 0s 14ms/step - loss: 2259308.5000 - val_loss: 6690576.0000
Epoch 288/10000
1/1 [==============================] - 0s 14ms/step - loss: 5126794.0000 - val_loss: 1802159.6250
Epoch 289/10000
1/1 [==============================] - 0s 14ms/step - loss: 2879135.7500 - val_loss: 2013893.3750
Epoch 290/10000
1/1 [==============================] - 0s 14ms/step - loss: 3681779.7500 - val_loss: 2644433.5000
Epoch 291/10000
1/1 [==============================] - 0s 14ms/step - loss: 3346648.5000 - val_loss: 1598720.6250
Epoch 292/10000
1/1 [==============================] - 0s 14ms/step - loss: 2214576.5000 - val_loss: 1835081.0000
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 2527636.7500 - val_loss: 1157653.8750
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 2374002.0000 - val_loss: 735

...

Epoch 1582/10000
1/1 [==============================] - 0s 15ms/step - loss: 1936416.7500 - val_loss: 137514.9219
Epoch 1583/10000
1/1 [==============================] - 0s 13ms/step - loss: 2024030.6250 - val_loss: 281075.2188
Epoch 1584/10000
1/1 [==============================] - 0s 13ms/step - loss: 1893847.1250 - val_loss: 343636.1875
Epoch 1585/10000
1/1 [==============================] - 0s 13ms/step - loss: 2524701.5000 - val_loss: 222603.6250
Epoch 1586/10000
1/1 [==============================] - 0s 14ms/step - loss: 2236195.2500 - val_loss: 183219.3281
Epoch 1587/10000
1/1 [==============================] - 0s 14ms/step - loss: 2078676.0000 - val_loss: 426691.2812
Epoch 1588/10000
1/1 [==============================] - 0s 14ms/step - loss: 2198136.5000 - val_loss: 169096.2969
Epoch 1589/10000
1/1 [==============================] - 0s 14ms/step - loss: 2590008.7500 - val_loss: 246798.9219
Epoch 1590/10000
1/1 [==============================] - 0s 13ms/step - loss: 1307998.250

Epoch 1654/10000
1/1 [==============================] - 0s 13ms/step - loss: 1885773.5000 - val_loss: 550726.0625
Epoch 1655/10000
1/1 [==============================] - 0s 14ms/step - loss: 2375270.0000 - val_loss: 546210.5625
Epoch 1656/10000
1/1 [==============================] - 0s 14ms/step - loss: 1960623.8750 - val_loss: 451088.5312
Epoch 1657/10000
1/1 [==============================] - 0s 14ms/step - loss: 1672629.3750 - val_loss: 648431.4375
Epoch 1658/10000
1/1 [==============================] - 0s 14ms/step - loss: 2084174.1250 - val_loss: 491952.7188
Epoch 1659/10000
1/1 [==============================] - 0s 14ms/step - loss: 2174309.5000 - val_loss: 627114.6875
Epoch 1660/10000
1/1 [==============================] - 0s 14ms/step - loss: 2146717.2500 - val_loss: 360071.0000
Epoch 1661/10000
1/1 [==============================] - 0s 13ms/step - loss: 1809590.5000 - val_loss: 540641.7500
Epoch 1662/10000
1/1 [==============================] - 0s 13ms/step - loss: 1625884.250

Epoch 1726/10000
1/1 [==============================] - 0s 14ms/step - loss: 2347857.0000 - val_loss: 296240.0312
Epoch 1727/10000
1/1 [==============================] - 0s 14ms/step - loss: 1298167.8750 - val_loss: 585070.9375
Epoch 1728/10000
1/1 [==============================] - 0s 14ms/step - loss: 2022070.1250 - val_loss: 201591.4375
Epoch 1729/10000
1/1 [==============================] - 0s 14ms/step - loss: 1320239.2500 - val_loss: 419404.6562
Epoch 1730/10000
1/1 [==============================] - 0s 14ms/step - loss: 2524180.5000 - val_loss: 619820.8125
Epoch 1731/10000
1/1 [==============================] - 0s 14ms/step - loss: 1767452.0000 - val_loss: 142039.1094
Epoch 1732/10000
1/1 [==============================] - 0s 14ms/step - loss: 1705236.5000 - val_loss: 382427.1562
Epoch 1733/10000
1/1 [==============================] - 0s 14ms/step - loss: 2785785.5000 - val_loss: 675300.5625
Epoch 1734/10000
1/1 [==============================] - 0s 14ms/step - loss: 2075659.000

Epoch 1798/10000
1/1 [==============================] - 0s 14ms/step - loss: 2953577.5000 - val_loss: 338362.7500
Epoch 1799/10000
1/1 [==============================] - 0s 14ms/step - loss: 1871215.3750 - val_loss: 627859.3125
Epoch 1800/10000
1/1 [==============================] - 0s 13ms/step - loss: 2542903.0000 - val_loss: 448168.8438
Epoch 1801/10000
1/1 [==============================] - 0s 14ms/step - loss: 2821146.7500 - val_loss: 408127.2500
Epoch 1802/10000
1/1 [==============================] - 0s 14ms/step - loss: 1417692.3750 - val_loss: 465591.3438
Epoch 1803/10000
1/1 [==============================] - 0s 14ms/step - loss: 2373332.5000 - val_loss: 372540.6562
Epoch 1804/10000
1/1 [==============================] - 0s 14ms/step - loss: 1903966.8750 - val_loss: 566570.5625
Epoch 1805/10000
1/1 [==============================] - 0s 14ms/step - loss: 2535157.2500 - val_loss: 339050.6562
Epoch 1806/10000
1/1 [==============================] - 0s 14ms/step - loss: 1893267.750

Epoch 1870/10000
1/1 [==============================] - 0s 15ms/step - loss: 1705734.5000 - val_loss: 189203.4844
Epoch 1871/10000
1/1 [==============================] - 0s 14ms/step - loss: 1840067.0000 - val_loss: 357946.5938
Epoch 1872/10000
1/1 [==============================] - 0s 14ms/step - loss: 1939221.7500 - val_loss: 181963.0781
Epoch 1873/10000
1/1 [==============================] - 0s 14ms/step - loss: 1569814.7500 - val_loss: 182212.7500
Epoch 1874/10000
1/1 [==============================] - 0s 14ms/step - loss: 1289076.8750 - val_loss: 205437.3125
Epoch 1875/10000
1/1 [==============================] - 0s 14ms/step - loss: 1796783.7500 - val_loss: 254733.5156
Epoch 1876/10000
1/1 [==============================] - 0s 14ms/step - loss: 1499895.0000 - val_loss: 367738.8750
Epoch 1877/10000
1/1 [==============================] - 0s 14ms/step - loss: 1843562.0000 - val_loss: 298787.4375
Epoch 1878/10000
1/1 [==============================] - 0s 14ms/step - loss: 1830758.625

In [22]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[12113.838](test_target) - [[10024.9375]](prediction) = 2088.9004999999997


In [23]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [24]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
0,-1.435135,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.700607,-1.593377,-1.575984,-1.980026,-1.942028,1.882710,-0.069679
1,-1.199105,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.108050,-1.060432,-1.107444,-1.186383,-1.098773,1.001093,1.458929
2,-0.724190,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.570105,-0.559276,-0.651569,-0.292900,-0.301154,0.712594,1.493043
3,-0.163232,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.085751,-0.123447,-0.265127,0.301776,0.218363,-0.316961,0.383215
4,0.285380,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.346036,0.257438,0.373867,0.761640,0.571261,-0.618515,-0.656508
5,0.639974,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.726276,0.554545,0.746270,0.796478,0.594785,-0.977369,-1.225666
6,1.086002,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.055993,0.788918,1.132417,0.771469,0.600128,-0.629670,-1.239651
7,1.510307,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.336208,1.735632,1.347570,0.827948,1.357418,-1.053882,-0.143683


train_target:


,São Paulo - Consumo de Cimento (t)
0,9148.318
1,9724.722
2,10863.790
3,11775.453
4,13476.268
5,13304.301
6,14534.980
7,14871.943


test_input:


...

,São Paulo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - IDH,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - Valor Cimento Portland
17,-0.733293,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.064924,0.776026,-1.154968,-1.420169,0.021261,0.852494,2.586291


test_target:


,São Paulo - Consumo de Cimento (t)
17,12113.838


1/1 [==============================] - 0s 35ms/step
Error: 2088.9004999999997




[14359.2666015625,
 14763.990234375,
 15077.427734375,
 13801.8232421875,
 13336.740234375,
 10815.48046875,
 10232.044921875,
 9907.9130859375,
 9992.17578125,
 10024.9375]

In [25]:
display(targets)
display(predictions)

[15159.683,
 15089.297,
 13714.511285714289,
 11767.65342857143,
 9811.084285714289,
 9918.501,
 10001.077,
 10458.276,
 11181.809,
 12113.838]

[14359.2666015625,
 14763.990234375,
 15077.427734375,
 13801.8232421875,
 13336.740234375,
 10815.48046875,
 10232.044921875,
 9907.9130859375,
 9992.17578125,
 10024.9375]

In [26]:
mae = mean_absolute_error(predictions, targets)
mae

1300.5309398437494

In [27]:
porcentage = mae/np.mean(targets)
porcentage

0.10909054869216918